# Advanced RAG With Re-rank

In [ ]:
#!pip install -U huggingface_hub
#!pip install llama-index
#!pip install langchain-community
#!pip install langchain-core
#!pip install llama-index-core

In [3]:
from llama_index.core import SimpleDirectoryReader
import torch

input_dir_path = r"D:/Courses/RAG Course/"
#Next code will read all pdf files in the directory

In [4]:
# load data
loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            required_exts=[".pdf"],
            recursive=True
        )
docs = loader.load_data()

In [5]:
docs

[Document(id_='6f2a0321-5dc7-48e1-9ff3-1c983afb7cce', embedding=None, metadata={'page_label': '1', 'file_name': 'Q-Lora.pdf', 'file_path': 'D:\\Courses\\RAG Course\\Q-Lora.pdf', 'file_type': 'application/pdf', 'file_size': 1110975, 'creation_date': '2024-07-18', 'last_modified_date': '2024-07-18'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='QL ORA: Efficient Finetuning of Quantized LLMs\nTim Dettmers∗Artidoro Pagnoni∗Ari Holtzman\nLuke Zettlemoyer\nUniversity of Washington\n{dettmers,artidoro,ahai,lsz}@cs.washington.edu\nAbstract\nWe present QLORA, an efficient finetuning approach that reduces memory us-\nage enough to finetune a 65B parameter model on a single 48GB GPU while\npreserving full 16-bit finetuning task performance. QLORAbackpropagates g

In [6]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter()

nodes = parser.get_nodes_from_documents(docs)

In [7]:
nodes[0].text

'QL ORA: Efficient Finetuning of Quantized LLMs\nTim Dettmers∗Artidoro Pagnoni∗Ari Holtzman\nLuke Zettlemoyer\nUniversity of Washington\n{dettmers,artidoro,ahai,lsz}@cs.washington.edu\nAbstract\nWe present QLORA, an efficient finetuning approach that reduces memory us-\nage enough to finetune a 65B parameter model on a single 48GB GPU while\npreserving full 16-bit finetuning task performance. QLORAbackpropagates gradi-\nents through a frozen, 4-bit quantized pretrained language model into Low Rank\nAdapters (LoRA). Our best model family, which we name Guanaco , outperforms\nall previous openly released models on the Vicuna benchmark, reaching 99.3%\nof the performance level of ChatGPT while only requiring 24 hours of finetuning\non a single GPU. QLORAintroduces a number of innovations to save memory\nwithout sacrificing performance: (a) 4-bit NormalFloat (NF4), a new data type that\nis information theoretically optimal for normally distributed weights (b) Double\nQuantization to reduce

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
#%pip install git+https://github.com/huggingface/transformers
from transformers import BitsAndBytesConfig

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers to c:\users\siddhanna.janai\appdata\local\temp\pip-req-build-uhpqy_h2
  Resolved https://github.com/huggingface/transformers to commit 673d30b8268b584f9c600be27e048de0c714c0af
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\siddhanna.janai\AppData\Local\Temp\pip-req-build-uhpqy_h2'

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from getpass import getpass
#from langchain import HuggingFaceHub
from langchain_community.llms import HuggingFaceHub
import os

HUGGINGFACEHUB_API_TOKEN = getpass()
#place your huggingface API key after running this cell
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN


 ········


In [12]:
llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-alpha", #"meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 256,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

C:\Users\siddhanna.janai\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


In [28]:
#pip install pydantic

In [13]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

def messages_to_prompt(messages):
  prompt = ""
  for message in messages:
    if message.role == 'system':
      prompt += f"<|system|>\n{message.content}</s>\n"
    elif message.role == 'user':
      prompt += f"<|user|>\n{message.content}</s>\n"
    elif message.role == 'assistant':
      prompt += f"<|assistant|>\n{message.content}</s>\n"

  # ensure we start with a system prompt, insert blank if needed
  if not prompt.startswith("<|system|>\n"):
    prompt = "<|system|>\n</s>\n" + prompt

  # add final assistant prompt
  prompt = prompt + "<|assistant|>\n"

  return prompt



In [15]:
from getpass import getpass

GOOGLE_API_KEY = getpass("Enter your Google API:")

Enter your Google API: ········


In [35]:
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
import torch

Settings.embed_model = GeminiEmbedding(model_name="models/embedding-001", api_key=GOOGLE_API_KEY)
Settings.llm = Gemini(model="models/gemini-1.5-pro",api_key=GOOGLE_API_KEY,messages_to_prompt=messages_to_prompt)

In [36]:
# ServiceContext
from llama_index.core import ServiceContext
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor


service_context = ServiceContext.from_defaults(llm=Settings.llm,
                                               embed_model=Settings.embed_model
                                               )

# index
vector_index = VectorStoreIndex(
    nodes, service_context=service_context
)

# configure retriever
retriever = VectorIndexRetriever(
    index=vector_index,
    similarity_top_k=5,
    service_context=service_context)

In [37]:
#%pip install llama-index 
#%pip install llama-index-postprocessor-cohere-rerank 

In [38]:
# Define all embeddings and rerankers
from llama_index.postprocessor.cohere_rerank import CohereRerank

from llama_index.core.postprocessor import SentenceTransformerRerank
RERANKERS = {
    "WithoutReranker": "None",
    #"CohereRerank": CohereRerank(api_key="FmQZiK9b6m6Iypd7GGHvyi8oduAEuYvqsYNaBWf3", top_n=5),
    "bge-reranker-base": SentenceTransformerRerank(model="BAAI/bge-reranker-base", top_n=5),
    "bge-reranker-large": SentenceTransformerRerank(model="BAAI/bge-reranker-large", top_n=5)
}

In [39]:
# helper functions

from copy import deepcopy
from IPython.core.display import display, HTML
import pandas as pd

def get_retrieved_nodes(
    query_str, reranker
):
    query_bundle = QueryBundle(query_str)

    retrieved_nodes = retriever.retrieve(query_bundle)

    if reranker != "None":
      retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
    else:
        retrieved_nodes

    return retrieved_nodes


def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "<br>")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        node = deepcopy(node)
        node.node.metadata = {}
        node_text = node.node.get_text()
        node_text = node_text.replace("\n", " ")

        result_dict = {"Score": node.score, "Text": node_text}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

In [40]:
#pip install llama-index[query_tools]

In [41]:
from llama_index.core.schema import QueryBundle


In [42]:
query_str = "Different finetuning methods and their memory requirements"

# Loop over rerankers
for rerank_name, reranker in RERANKERS.items():
    print(f"Running Evaluation for Reranker: {rerank_name}")

    query_bundle = QueryBundle(query_str)

    retrieved_nodes = retriever.retrieve(query_bundle)

    if reranker != "None":
      retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
    else:
        retrieved_nodes

    print(f"Visualize Retrieved Nodes for Reranker: {rerank_name}")
    visualize_retrieved_nodes(retrieved_nodes)

Running Evaluation for Reranker: WithoutReranker
Visualize Retrieved Nodes for Reranker: WithoutReranker


,Score,Text
0,0.697117,"[72] A. Zeng, X. Liu, Z. Du, Z. Wang, H. Lai, M. Ding, Z. Yang, Y . Xu, W. Zheng, X. Xia, et al. Glm-130b: An open bilingual pre-trained model. arXiv preprint arXiv:2210.02414 , 2022. [73] S. Zhang, S. Roller, N. Goyal, M. Artetxe, M. Chen, S. Chen, C. Dewan, M. Diab, X. Li, X. V . Lin, et al. Opt: Open pre-trained transformer language models. arXiv preprint arXiv:2205.01068 , 2022. [74] R. Zhong, K. Lee, Z. Zhang, and D. Klein. Adapting language models for zero-shot learning by meta-tuning on dataset and prompt collections. arXiv preprint arXiv:2104.04670 , 2021. 15"
1,0.693002,"LLaMA model size0%25%50%75%100% 7B (6.9 GB) 13B (11.3 GB) 33B (24.7 GB) 65B (45.0 GB)Input gradient Optimizer Weight gradient Adapters ModelFigure 8: Breakdown of the memory footprint of different LLaMA models. The input gradient size is for batch size 1 and sequence length 512 and is estimated only for adapters and the base model weights (no attention). Numbers on the bars are memory footprint in MB of individual elements of the total footprint. While some models do not quite fit on certain GPUs, paged optimizers provide enough memory to allow these models to fit. 28"
2,0.677866,"Chiang, Z. Li, Z. Lin, Y . Sheng, Z. Wu, H. Zhang, L. Zheng, S. Zhuang, Y . Zhuang, J. E. Gonzalez, I. Stoica, and E. P. Xing. Vicuna: An open-source chatbot impressing gpt-4 with 90%* chatgpt quality, March 2023. URL https://lmsys.org/blog/2023-03-30-vicuna/ . [11] P. F. Christiano, J. Leike, T. Brown, M. Martic, S. Legg, and D. Amodei. Deep reinforcement learning from human preferences. Advances in neural information processing systems , 30, 2017. [12] H. W. Chung, L. Hou, S. Longpre, B. Zoph, Y . Tay, W. Fedus, E. Li, X. Wang, M. De- hghani, S. Brahma, et al. Scaling instruction-finetuned language models. arXiv preprint arXiv:2210.11416 , 2022. [13] T. Dettmers and L. Zettlemoyer. The case for 4-bit precision: k-bit inference scaling laws. arXiv preprint arXiv:2212.09720 , 2022. [14] T. Dettmers, M. Lewis, Y . Belkada, and L. Zettlemoyer. LLM.int8(): 8-bit matrix multiplication for transformers at scale. Advances in Neural Information Processing Systems 35: Annual Conference on Neural Information Processing Systems 2022, NeurIPS 2022 , 2022. [15] T. Dettmers, M. Lewis, S. Shleifer, and L. Zettlemoyer. 8-bit optimizers via block-wise quantization. 9th International Conference on Learning Representations, ICLR , 2022. [16] A. E. Elo. The proposed uscf rating system. its development, theory, and applications. Chess Life, 22(8):242–247, 1967. [17] A. E. Elo. The rating of chessplayers, past and present . Arco Pub., 1978. 11"
3,0.672513,"Figure 1: Different finetuning methods and their memory requirements. QLORAimproves over LoRA by quantizing the transformer model to 4-bit precision and using paged optimizers to handle memory spikes. 2 Background Block-wise k-bit Quantization Quantization is the process of discretizing an input from a rep- resentation that holds more information to a representation with less information. It often means taking a data type with more bits and converting it to fewer bits, for example from 32-bit floats to 8-bit Integers. To ensure that the entire range of the low-bit data type is used, the input data type is commonly rescaled into the target data type range through normalization by the absolute maximum of the input elements, which are usually structured as a tensor. For example, quantizing a 32-bit Floating Point (FP32) tensor into a Int8 tensor with range [−127,127]: XInt8=round127 absmax (XFP32)XFP32 =round (cFP32·XFP32), (1) where cis the quantization constant orquantization scale . Dequantization is the inverse: dequant (cFP32,XInt8) =XInt8 cFP32=XFP32(2) The problem with this approach is that if a large magnitude value (i.e., an outlier) occurs in the input tensor, then the quantization bins—certain bit combinations—are not utilized well with few or no numbers quantized in some bins. To prevent the outlier issue, a 

Running Evaluation for Reranker: CohereRerank
Visualize Retrieved Nodes for Reranker: CohereRerank


,Score,Text
0,0.998545,"Figure 1: Different finetuning methods and their memory requirements. QLORAimproves over LoRA by quantizing the transformer model to 4-bit precision and using paged optimizers to handle memory spikes. 2 Background Block-wise k-bit Quantization Quantization is the process of discretizing an input from a rep- resentation that holds more information to a representation with less information. It often means taking a data type with more bits and converting it to fewer bits, for example from 32-bit floats to 8-bit Integers. To ensure that the entire range of the low-bit data type is used, the input data type is commonly rescaled into the target data type range through normalization by the absolute maximum of the input elements, which are usually structured as a tensor. For example, quantizing a 32-bit Floating Point (FP32) tensor into a Int8 tensor with range [−127,127]: XInt8=round127 absmax (XFP32)XFP32 =round (cFP32·XFP32), (1) where cis the quantization constant orquantization scale . Dequantization is the inverse: dequant (cFP32,XInt8) =XInt8 cFP32=XFP32(2) The problem with this approach is that if a large magnitude value (i.e., an outlier) occurs in the input tensor, then the quantization bins—certain bit combinations—are not utilized well with few or no numbers quantized in some bins. To prevent the outlier issue, a common approach is to chunk the input tensor into blocks that are independently quantized, each with their own quantization constant c. This can be formalized as follows: We chunk the input tensor X∈Rb×hintoncontiguous blocks of sizeBby flattening the input tensor and slicing the linear segment into n= (b×h)/Bblocks. We quantize these blocks independently with Equation 1 to create a quantized tensor and nquantization constants ci. Low-rank Adapters Low-rank Adapter (LoRA) finetuning [ 28] is a method that reduces memory requirements by using a small set of trainable parameters, often termed adapters, while not updating the full model parameters which remain fixed. Gradients during stochastic gradient descent are passed through the fixed pretrained model weights to the adapter, which is updated to optimize the loss function. LoRA augments a linear projection through an additional factorized projection. Given a projection XW =YwithX∈Rb×h,W∈Rh×oLoRA computes: Y=XW +sXL 1L2, (3) whereL1∈Rh×randL2∈Rr×o, and sis a scalar. 3 QL ORA Finetuning QLORAachieves high-fidelity 4-bit finetuning via two techniques we propose—4-bit NormalFloat (NF4) quantization and Double Quantization. Additionally, we introduce Paged Optimizers, to 3"
1,0.947769,"quantization. On average, for a blocksize of 64, this quantization reduces the memory footprint per parameter from 32/64 = 0 .5bits, to 8/64 + 32 /(64·256) = 0 .127bits, a reduction of 0.373 bits per parameter. Paged Optimizers use the NVIDIA unified memory3feature which does automatic page-to-page transfers between the CPU and GPU for error-free GPU processing in the scenario where the GPU occasionally runs out-of-memory. The feature works like regular memory paging between CPU RAM and the disk. We use this feature to allocate paged memory for the optimizer states which are then automatically evicted to CPU RAM when the GPU runs out-of-memory and paged back into GPU memory when the memory is needed in the optimizer update step. QL ORA.Using the components described above, we define QLORAfor a single linear layer in the quantized base model with a single LoRA adapter as follows: YBF16=XBF16doubleDequant (cFP32 1, ck-bit 2,WNF4) +XBF16LBF16 1LBF16 2, (5) doubleDequant (cFP32 1, ck-bit 2,Wk-bit) =dequant (dequant (cFP32 1, ck-bit 2),W4bit) =WBF16,(6) We use NF4 for Wand FP8 for c2. We use a blocksize of 64 for Wfor higher quantization precision and a blocksize of 256 for c2to conserve memory. For parameter updates only the gradient with respect to the error for the adapters weights∂E ∂Liare needed, and not for 4-bit weights∂E ∂W. However, the calculation of∂E ∂Li

Running Evaluation for Reranker: bge-reranker-base
Visualize Retrieved Nodes for Reranker: bge-reranker-base


,Score,Text
0,0.998165,"Figure 1: Different finetuning methods and their memory requirements. QLORAimproves over LoRA by quantizing the transformer model to 4-bit precision and using paged optimizers to handle memory spikes. 2 Background Block-wise k-bit Quantization Quantization is the process of discretizing an input from a rep- resentation that holds more information to a representation with less information. It often means taking a data type with more bits and converting it to fewer bits, for example from 32-bit floats to 8-bit Integers. To ensure that the entire range of the low-bit data type is used, the input data type is commonly rescaled into the target data type range through normalization by the absolute maximum of the input elements, which are usually structured as a tensor. For example, quantizing a 32-bit Floating Point (FP32) tensor into a Int8 tensor with range [−127,127]: XInt8=round127 absmax (XFP32)XFP32 =round (cFP32·XFP32), (1) where cis the quantization constant orquantization scale . Dequantization is the inverse: dequant (cFP32,XInt8) =XInt8 cFP32=XFP32(2) The problem with this approach is that if a large magnitude value (i.e., an outlier) occurs in the input tensor, then the quantization bins—certain bit combinations—are not utilized well with few or no numbers quantized in some bins. To prevent the outlier issue, a common approach is to chunk the input tensor into blocks that are independently quantized, each with their own quantization constant c. This can be formalized as follows: We chunk the input tensor X∈Rb×hintoncontiguous blocks of sizeBby flattening the input tensor and slicing the linear segment into n= (b×h)/Bblocks. We quantize these blocks independently with Equation 1 to create a quantized tensor and nquantization constants ci. Low-rank Adapters Low-rank Adapter (LoRA) finetuning [ 28] is a method that reduces memory requirements by using a small set of trainable parameters, often termed adapters, while not updating the full model parameters which remain fixed. Gradients during stochastic gradient descent are passed through the fixed pretrained model weights to the adapter, which is updated to optimize the loss function. LoRA augments a linear projection through an additional factorized projection. Given a projection XW =YwithX∈Rb×h,W∈Rh×oLoRA computes: Y=XW +sXL 1L2, (3) whereL1∈Rh×randL2∈Rr×o, and sis a scalar. 3 QL ORA Finetuning QLORAachieves high-fidelity 4-bit finetuning via two techniques we propose—4-bit NormalFloat (NF4) quantization and Double Quantization. Additionally, we introduce Paged Optimizers, to 3"
1,0.066596,"The main question now is whether QLoRA can perform as well as full-model finetuning. Furthermore, we want to analyze the components of QLoRA including the impact of NormalFloat4 over standard Float4. The follow- ing sections will discuss the experiments that aimed at answering these questions. Experimental setup. We consider three architec- tures (encoder, encoder-decoder, and decoder only) and compare QLoRA with 16-bit adapter-finetuning and with full-finetuning for models up to 3B. Our evaluations include GLUE [ 59] with RoBERTa- large [ 38], Super-NaturalInstructionss [ 62] with T5 [50], and 5-shot MMLU [ 24] after finetuning LLaMA on Flan v2 [39] and Alpaca [56]. To additionally study the advantages of NF4 over other 4-bit data types, we use the setup of Dettmers and Zettlemoyer [13] and measure post-quantization zero-shot accuracy and perplexity across different models (OPT [ 73], LLaMA [ 58], BLOOM [ 53], Pythia [ 7]) for model sizes 125m - 13B. We provide more details in the results section for each particular setup to make the results more readable. Full details in Appendix C. While paged optimizers are critical to do 33B/65B QLORAtuning on a single 24/48GB GPU, we do not provide hard measurements for Paged Optimizers since the paging only occurs when processing 3https://docs.nvidia.com/cuda/cuda-c-programming-guide 5"
2,0.056656,"[37] H. Liu, D. Tam, M

Running Evaluation for Reranker: bge-reranker-large
Visualize Retrieved Nodes for Reranker: bge-reranker-large


,Score,Text
0,0.995404,"Figure 1: Different finetuning methods and their memory requirements. QLORAimproves over LoRA by quantizing the transformer model to 4-bit precision and using paged optimizers to handle memory spikes. 2 Background Block-wise k-bit Quantization Quantization is the process of discretizing an input from a rep- resentation that holds more information to a representation with less information. It often means taking a data type with more bits and converting it to fewer bits, for example from 32-bit floats to 8-bit Integers. To ensure that the entire range of the low-bit data type is used, the input data type is commonly rescaled into the target data type range through normalization by the absolute maximum of the input elements, which are usually structured as a tensor. For example, quantizing a 32-bit Floating Point (FP32) tensor into a Int8 tensor with range [−127,127]: XInt8=round127 absmax (XFP32)XFP32 =round (cFP32·XFP32), (1) where cis the quantization constant orquantization scale . Dequantization is the inverse: dequant (cFP32,XInt8) =XInt8 cFP32=XFP32(2) The problem with this approach is that if a large magnitude value (i.e., an outlier) occurs in the input tensor, then the quantization bins—certain bit combinations—are not utilized well with few or no numbers quantized in some bins. To prevent the outlier issue, a common approach is to chunk the input tensor into blocks that are independently quantized, each with their own quantization constant c. This can be formalized as follows: We chunk the input tensor X∈Rb×hintoncontiguous blocks of sizeBby flattening the input tensor and slicing the linear segment into n= (b×h)/Bblocks. We quantize these blocks independently with Equation 1 to create a quantized tensor and nquantization constants ci. Low-rank Adapters Low-rank Adapter (LoRA) finetuning [ 28] is a method that reduces memory requirements by using a small set of trainable parameters, often termed adapters, while not updating the full model parameters which remain fixed. Gradients during stochastic gradient descent are passed through the fixed pretrained model weights to the adapter, which is updated to optimize the loss function. LoRA augments a linear projection through an additional factorized projection. Given a projection XW =YwithX∈Rb×h,W∈Rh×oLoRA computes: Y=XW +sXL 1L2, (3) whereL1∈Rh×randL2∈Rr×o, and sis a scalar. 3 QL ORA Finetuning QLORAachieves high-fidelity 4-bit finetuning via two techniques we propose—4-bit NormalFloat (NF4) quantization and Double Quantization. Additionally, we introduce Paged Optimizers, to 3"
1,0.042283,"The main question now is whether QLoRA can perform as well as full-model finetuning. Furthermore, we want to analyze the components of QLoRA including the impact of NormalFloat4 over standard Float4. The follow- ing sections will discuss the experiments that aimed at answering these questions. Experimental setup. We consider three architec- tures (encoder, encoder-decoder, and decoder only) and compare QLoRA with 16-bit adapter-finetuning and with full-finetuning for models up to 3B. Our evaluations include GLUE [ 59] with RoBERTa- large [ 38], Super-NaturalInstructionss [ 62] with T5 [50], and 5-shot MMLU [ 24] after finetuning LLaMA on Flan v2 [39] and Alpaca [56]. To additionally study the advantages of NF4 over other 4-bit data types, we use the setup of Dettmers and Zettlemoyer [13] and measure post-quantization zero-shot accuracy and perplexity across different models (OPT [ 73], LLaMA [ 58], BLOOM [ 53], Pythia [ 7]) for model sizes 125m - 13B. We provide more details in the results section for each particular setup to make the results more readable. Full details in Appendix C. While paged optimizers are critical to do 33B/65B QLORAtuning on a single 24/48GB GPU, we do not provide hard measurements for Paged Optimizers since the paging only occurs when processing 3https://docs.nvidia.com/cuda/cuda-c-programming-guide 5"
2,0.035546,Table 4: Zero-shot Vicu

In [ ]:
#from llama_index.core.response.notebook_utils import display_source_node

#for node in retrieved_nodes:
    #display_source_node(node, source_length=1000)

In [44]:
#pip install -U huggingface_hub

For the following code visit the path File ~\AppData\Roaming\Python\Python311\site-packages\llama_index\core\llama_dataset\legacy\embedding.py:87 >> open in notepad and change
#####  response = llm.complete(query) to response=llm.invoke(query)

File ~\AppData\Roaming\Python\Python311\site-packages\llama_index\core\llama_dataset\legacy\embedding.py:87, in generate_qa_embedding_pairs(nodes, llm, qa_generate_prompt_tmpl, num_questions_per_chunk)
     83 for node_id, text in tqdm(node_dict.items()):
     84     query = qa_generate_prompt_tmpl.format(
     85         context_str=text, num_questions_per_chunk=num_questions_per_chunk
     86     )

---> 87     response = llm.complete(query)

In [45]:
# Prompt to generate questions
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
qa_generate_prompt_tmpl = """\
Context information is below.

---------------------
{context_str}
---------------------

Given the context information and not prior knowledge.
generate only questions based on the below query.

You are a Professor. Your task is to setup \
{num_questions_per_chunk} questions for an upcoming \
quiz/examination. The questions should be diverse in nature \
across the document. The questions should not contain options, not start with Q1/ Q2. \
Restrict the questions to the context information provided.\
"""

# Evaluator

qa_dataset = generate_question_context_pairs(nodes, llm=llm, num_questions_per_chunk=2, qa_generate_prompt_tmpl=qa_generate_prompt_tmpl)

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:12<00:00,  3.31it/s]


In [1]:
#qa_dataset

In [46]:
# helper function for displaying results
def display_results(reranker_name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame({"Reranker": [reranker_name], "hit_rate": [hit_rate], "mrr": [mrr]})

    return metric_df

In [48]:
from llama_index.core.evaluation import RetrieverEvaluator
query_str = "Different finetuning methods and their memory requirements?"

results_df = pd.DataFrame()
# Loop over rerankers
for rerank_name, reranker in RERANKERS.items():
    print(f"Running Evaluation for Reranker: {rerank_name}")

    query_bundle = QueryBundle(query_str)

    retrieved_nodes = retriever.retrieve(query_bundle)

    if reranker != "None":
      retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
    else:
        retrieved_nodes

    retriever_evaluator = RetrieverEvaluator.from_metric_names(
        ["mrr", "hit_rate"], retriever=retriever
    )

    eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

    current_df = display_results(rerank_name, eval_results)
    results_df = pd.concat([results_df, current_df], ignore_index=True)

Running Evaluation for Reranker: WithoutReranker
Running Evaluation for Reranker: CohereRerank
Running Evaluation for Reranker: bge-reranker-base
Running Evaluation for Reranker: bge-reranker-large


In [50]:
results_df

,Reranker,hit_rate,mrr
0,WithoutReranker,0.25,0.073224
1,CohereRerank,0.25,0.073224
2,bge-reranker-base,0.25,0.073224
3,bge-reranker-large,0.25,0.073224


In [52]:
#eval_results

# Only Querying with retriever

In [ ]:
retrieved_nodes = retriever.retrieve(query_str)

In [ ]:
from llama_index.core import PromptTemplate
qa_prompt = PromptTemplate(
    """\
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: \
"""
)

In [ ]:
def generate_response(retrieved_nodes, query_str, qa_prompt, llm):
    context_str = "\n\n".join([r.get_content() for r in retrieved_nodes])
    fmt_qa_prompt = qa_prompt.format(
        context_str=context_str, query_str=query_str
    )
    response = llm.invoke(fmt_qa_prompt)
    return str(response), fmt_qa_prompt

In [ ]:
response, fmt_qa_prompt = generate_response(retrieved_nodes, query_str, qa_prompt, llm)


In [ ]:
print(f"*****Response******:\n{response}\n\n")